# Tutorial 3.  Interactive Visualization in Blender


<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons Licence" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" title='This work is licensed under a Creative Commons Attribution 4.0 International License.' align="right"/></a>

Authors: 

- Dr Yuxuan Zhuang - yuxuan.zhuang@stanford.edu 

This notebook is used for MDAnalysis Workshop, February 2024. As most of the APIs are not stable, please check the changes in the latest version of MolecularNodes before using this notebook.


**Note**:
This notebook cannot be run on Google Colab.
To run it, you will need to have a local installation of [Blender](https://www.blender.org/download/), [MolecularNodes](https://github.com/BradyAJohnston/MolecularNodes), and [BNoteBooks](https://github.com/BradyAJohnston/bnotebooks). See install instructions in INSTALL.md.

## **Jupyter cheat sheet**:
- to run the currently highlighted cell, hold <kbd>&#x21E7; Shift</kbd> and press <kbd>&#x23ce; Enter</kbd>;
- to get help for a specific function, place the cursor within the function's brackets, hold <kbd>&#x21E7; Shift</kbd>, and press <kbd>&#x21E5; Tab</kbd>;


## Learning outcomes:

* Understand how to use BNoteBooks to create interactive visualizations in Blender.
* Understand how to use MolecularNodes to visualize molecular structures with MDAnalysis in an interactive way.
* Show how to use Blender for advanced visualization of analysis results.

## Table of Contents

1. [MDAnalysisSession](#mdasession)  
2. [Advanced visualization](#advanced)

### Imports

We start by importing the necessary libraries, setting up the Blender, and downloading the necessary files.

In [5]:
import os
import bpy
import numpy as np
import molecularnodes as mn
from molecularnodes.io.parse.mda import MDAnalysisSession

import MDAnalysis as mda

In [6]:
# set render engine to cycles and device to GPU if available
bpy.context.scene.render.engine = 'CYCLES'
# bpy.context.scene.cycles.device = "GPU"

In [7]:
# Load trajectory in MDAnalysis
from MDAnalysisData.datasets import fetch_adk_transitions_DIMS
adk = fetch_adk_transitions_DIMS.fetch_adk_equilibrium()

u = mda.Universe(adk.topology, adk.trajectores[1])

/Applications/Blender.app/Contents/Resources/4.0/python/lib/python3.10/site-packages/MDAnalysis/coordinates/DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"


## MDAnalysisSession
<a id='mdasession'></a>


We will use the `MDAnalysisSession` class to create a Blender scene. `Universe` and `Atomgroup` can be visualized in Blender using the `MDAnalysisSession` class. 

In [8]:
# Create an MDAnalysis session for interacting with Blender

mda_session = MDAnalysisSession()

In [9]:
# Show the trajectory in Blender

mda_session.show(u)

appending material


bpy.data.objects['atoms']

In [10]:
# Alternatively, show only the protein and other custom selections

custom_selections = {'resid_33': 'resid 33',
                     'resid_156': 'resid 156',
                    }
mda_session.show(u,
                 selection='protein',
                 name='protein',
                 custom_selections=custom_selections
                )

bpy.data.objects['protein']

In [11]:
# add position averager transformation

from MDAnalysis.transformations import PositionAverager

transformation = PositionAverager(10, check_reset=True)
u.trajectory.add_transformations(transformation)

In [12]:
# save the .blend file

file_path = "blender_and_mda.blend"

bpy.ops.wm.save_as_mainfile(filepath=file_path)

# the .blend file can be opened by
# file_path = "blender_and_mda.blend"
# bpy.ops.wm.open_mainfile(filepath=file_path)
# mda_session = bpy.context.scene.mda_session

Info: Total files 11 | Changed 11 | Failed 0
Info: Saved "blender_and_mda.blend"


{'FINISHED'}

## Advanced usage
<a id='advanced'></a>

The example provided demonstrates a prototype for visualizing interatomic distances using Blender.

It features the power of Blender to manipulate and visualize the data in a 3D environment. It will also be our ongoing effort to develop a more user-friendly and interactive visualization tool for analysis results in MDAnalysis.

In [13]:
from bpy.app.handlers import persistent

class Distance_in_Blender:
    """
    Dispaly distance between two atomgroups in Blender
    """
    def __init__(self, coord1_arr, coord2_arr, dist_arr, world_scale=0.01):
        self.coord1_arr = coord1_arr
        self.coord2_arr = coord2_arr
        self.dist_arr = dist_arr
        self.n_frames = len(dist_arr)
        self.world_scale = world_scale
        self.draw()
        bpy.app.handlers.frame_change_post.append(self._update_position_handler_wrapper())

    def draw(self, name='distance'):
        line_data = bpy.data.curves.new(name=name, type='CURVE')
        line_data.dimensions = '3D'
        line_object = bpy.data.objects.new(f"{name}_line", line_data)
        bpy.context.collection.objects.link(line_object)
        bpy.context.view_layer.objects.active = line_object

        text_data = bpy.data.curves.new(name=name, type='FONT')
        text_object = bpy.data.objects.new(f"{name}_text", text_data)
        bpy.context.collection.objects.link(text_object)
        
        text_data.size = 8 * self.world_scale
        text_data.align_x = 'CENTER'
        text_data.align_y = 'CENTER'
        
        line = line_data.splines.new('POLY')
        line.points.add(1)

        self.line_data = line_data
        self.line_object = line_object
        self.text_data = text_data
        self.text_object = text_object
        self.line = line
        self._update_trajectory(0)

        line.resolution_u = 4
        line.use_cyclic_u = False
        line.use_endpoint_u = True
        line.use_endpoint_v = True
        line.use_smooth = False
    
    @persistent
    def _update_trajectory(self, frame):
        coord1 = self.coord1_arr[frame]
        coord2 = self.coord2_arr[frame]
        dist = np.mean(self.dist_arr[frame])
        
        if frame < 0:
            return
        elif frame >= self.n_frames:
            return

        for point, coord in zip(self.line.points, [coord1, coord2]):
            coord = list(coord * self.world_scale)
            point.co = (coord[0], coord[1], coord[2], 1)
        self.text_data.body = f'{dist:.1f} Å'
        
        middle_coord = np.mean([coord1, coord2], axis=0)
        self.text_object.location = middle_coord * self.world_scale
        # rotate the text object to face the camera
        self.text_object.rotation_euler = bpy.context.scene.camera.rotation_euler

    @persistent
    def _update_position_handler_wrapper(self):
        def update_position_handler(scene):
            frame = scene.frame_current
            self._update_trajectory(frame)
        return update_position_handler

In [14]:
# run analysis

from MDAnalysis.analysis import distances

res_33_asp = u.select_atoms('resid 33')
res_156_arg = u.select_atoms('resid 156')

coord1_arr = []
coord2_arr = []
dist_arr = []
for ts in u.trajectory:
    com_res_33 = res_33_asp.center_of_mass()
    com_res_156 = res_156_arg.center_of_mass()
    dist_arr.append(distances.distance_array(com_res_33,
                                             com_res_156,
                                             box=u.dimensions))

    coord1_arr.append(com_res_33)
    coord2_arr.append(com_res_156)

/Applications/Blender.app/Contents/Resources/4.0/python/lib/python3.10/site-packages/MDAnalysis/transformations/positionaveraging.py:193: Warning: Cannot average position for non sequentialiterations. Averager will be reset.
  warnings.warn('Cannot average position for non sequential'


In [15]:
dist_render = Distance_in_Blender(coord1_arr,
                                  coord2_arr,
                                  dist_arr,
                                  world_scale=mda_session.world_scale)